In [184]:
import numpy as np

# TASK 1
Implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the following substitution function:
    $$f : y_j(j): 2v_i(j) \mod  p, \ j$$

In [185]:
def subkey_generator_default(key):

    k_1 = [key[0],key[2],key[4],key[6]] 
    k_2 = [key[0],key[1],key[2],key[6]] 
    k_3 = [key[0],key[3],key[4],key[6]] 
    k_4 = [key[0],key[3],key[5],key[6]] 
    k_5 = [key[0],key[2],key[5],key[6]] 
    k_6 = [key[2],key[4],key[4],key[6]] 

    return [k_1, k_2, k_3, k_4, k_5, k_6]

def subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter)

    return (w + k)%p



In [186]:
 
def timesTwo(v,p):

    return np.multiply(v,2)%p

def transposition(y):

    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])

def linear(z, p):

    z_matrix = z.reshape(2,4)

    param_matrix = np.matrix([[2,5],[1,7]])

    w_matrix = np.dot(param_matrix,z_matrix)%p

    w_array = np.asarray(w_matrix).flatten()

    return w_array
        




In [187]:
def encryption(u,k,p,round,substitution,transposition,subkey_sum,linear, subkey_generator):

    k_list = subkey_generator(k)

    for i in range (round):
        
        #print("round : ",i)
        v = subkey_sum(k_list[i],u,p)         
        #print("post subkey_sum",v)
        y = substitution(v,p)
        #print("post timesTwo",y)
        z = transposition(y)
        #print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        #print("post linear", u)
        
    return x


In [188]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p=11
r = 5

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

print(x)


[4 0 0 9 7 0 0 3]


# TASK 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field $\mathbb{F} = GF(p)$.

In [189]:
def inv_timesTwo(v,p):
    mul_inv = pow(2,-1,p)
    return np.multiply(v,mul_inv)%p


def inv_transposition(y):
    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])


def inv_subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter) #list

    return (w - k)%p


def inv_linear(z, p):
    z_matrix = z.reshape(2, 4)

    param_matrix = np.array([[2, 5], [1, 7]])

    #Calcolo dell'inversa della matrice modulo p
    det = int(np.round(np.linalg.det(param_matrix)))  #Determinante della matrice
    det_inv = pow(det, -1, p)  #Inverso del determinante modulo p
    adj_matrix = np.array([[param_matrix[1, 1], -param_matrix[0, 1]],
                           [-param_matrix[1, 0], param_matrix[0, 0]]])  #Matrice aggiunta
    param_matrix_inv = (det_inv * adj_matrix) % p  #Inversa modulo p

    w_matrix = np.dot(param_matrix_inv, z_matrix) % p

    w_array = np.asarray(w_matrix, dtype=int).flatten()

    return w_array


def decryption(x,k,p,round):
    k_list = subkey_generator_default(k)

    for i in range(round,-1,-1):
        print("round : ", i)

        z = inv_subkey_sum(k_list[i],x,p)            #v = u - k
        print("post subkey_sum",z)
        
        if i == 0:
            x = z
            break
        elif i != round:
            w = inv_linear(z, p)
            print("post linear", w)
        else:
            w = z

        y = inv_transposition(w)
        print("post transpo",y)

        x = inv_timesTwo(y,p)
        print("post timesTwo",x)
        
    return x

Test for inv_linear

In [190]:
a = linear(np.array([1,0,3,5,0,0,8,0]), p)  #change the array to test
print(a)
b = inv_linear(a, p)
print(b)

[ 2  0  2 10  1  0  4  5]
[1 0 3 5 0 0 8 0]


In [191]:
u = decryption(x,k,p,r)
print(u)

round :  5
post subkey_sum [4 0 0 9 7 0 0 3]
post transpo [4 0 0 9 3 0 0 7]
post timesTwo [ 2  0  0 10  7  0  0  9]
round :  4
post subkey_sum [ 1  0  0 10  6  0  0  9]
post linear [6 0 0 4 0 0 0 7]
post transpo [6 0 0 4 7 0 0 0]
post timesTwo [3 0 0 2 9 0 0 0]
round :  3
post subkey_sum [2 0 0 2 8 0 0 0]
post linear [2 0 0 4 4 0 0 1]
post transpo [2 0 0 4 1 0 0 4]
post timesTwo [1 0 0 2 6 0 0 2]
round :  2
post subkey_sum [0 0 0 2 5 0 0 2]
post linear [ 7  0  0  9  6  0  0 10]
post transpo [ 7  0  0  9 10  0  0  6]
post timesTwo [ 9  0  0 10  5  0  0  3]
round :  1
post subkey_sum [ 8  0  0 10  4  0  0  3]
post linear [4 0 0 0 0 0 0 2]
post transpo [4 0 0 0 2 0 0 0]
post timesTwo [2 0 0 0 1 0 0 0]
round :  0
post subkey_sum [1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]


Other tests for decryption

In [192]:
a = np.array([1,8,3,6,5,6,7,2]) #change plaintext to test
k_test = np.array([1,0,4,0,3,0,2,0]) #change key to test
x = encryption(u,k_test,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)

print(b)
c = decryption(b,k,p,r)
print(c)

[1 0 3 5 0 0 8 0]
round :  5
post subkey_sum [1 0 3 5 0 0 8 0]
post transpo [1 0 3 5 0 8 0 0]
post timesTwo [6 0 7 8 0 4 0 0]
round :  4
post subkey_sum [ 5  0  7  8 10  4  0  0]
post linear [ 2 10  3  5  9  7  9  4]
post transpo [ 2 10  3  5  4  9  7  9]
post timesTwo [ 1  5  7  8  2 10  9 10]
round :  3
post subkey_sum [ 0  5  7  8  1 10  9 10]
post linear [ 8  2  9  8 10  9  0  5]
post transpo [ 8  2  9  8  5  0  9 10]
post timesTwo [ 4  1 10  4  8  0 10  5]
round :  2
post subkey_sum [ 3  1 10  4  7  0 10  5]
post linear [7 2 1 4 0 6 6 8]
post transpo [7 2 1 4 8 6 6 0]
post timesTwo [9 1 6 2 4 3 3 0]
round :  1
post subkey_sum [8 1 6 2 3 3 3 0]
post linear [7 4 3 4 1 3 0 1]
post transpo [7 4 3 4 1 0 3 1]
post timesTwo [9 2 7 2 6 0 7 6]
round :  0
post subkey_sum [8 2 7 2 5 0 7 6]
[8 2 7 2 5 0 7 6]


TASK 3

Identify the overall linear relationship for this simplified AES-like cipher, that is find the
matrices A ∈ F^(ℓx×ℓk) and B ∈ Fℓx×ℓu such that
x = E(k, u) = Ak + Bu mod p
with all operations in the field F = GF(p).

In [193]:

e = np.eye(8, dtype=int)
k_a = np.zeros(8,int)
A = np.empty((8,0),int)

x = encryption(u,k,p,r,
               substitution=timesTwo,
               transposition=transposition,
               subkey_sum=subkey_sum,
               linear=linear,
               subkey_generator=subkey_generator_default)


for i in range (8):

    encrypted_value = encryption(k_a, np.asarray(e[i]).flatten(), p, r,
                                 substitution=timesTwo,
                                 transposition=transposition,
                                 subkey_sum=subkey_sum,
                                 linear=linear,
                                 subkey_generator=subkey_generator_default)
    
    A = np.hstack((A, encrypted_value.reshape(-1, 1)))

print(A)



k_b = np.eye(8, dtype=int)
e = np.zeros(8,int)
B = np.empty((8,0),int)

for i in range (8):

    encrypted_value = encryption(np.asarray(k_b[i]).flatten(),e,p,r,
                                 substitution=timesTwo,
                                 transposition=transposition,
                                 subkey_sum=subkey_sum,
                                 linear=linear,
                                 subkey_generator=subkey_generator_default)
    B = np.hstack((B, encrypted_value.reshape(-1, 1)))


print(B)

x = (np.dot(A,k) + np.dot(B,u))%p

print(x)

[[ 9  0  1  0  0  0  6  0]
 [ 0  8  6  1  3  9  0  0]
 [ 0  6  0  8  3 10  0  0]
 [ 6  0  0  0  0  0 10  0]
 [ 2  0  1  0  0  0  3  0]
 [ 0  1  8  3 10  6  0  0]
 [ 0 10  0  5  7  6  0  0]
 [ 3  0  0  0  0  0  3  0]]
[[6 0 0 3 3 0 0 0]
 [0 6 3 0 0 3 0 0]
 [0 3 6 0 0 0 3 0]
 [3 0 0 6 0 0 0 3]
 [5 0 0 0 4 0 0 8]
 [0 5 0 0 0 4 8 0]
 [0 0 5 0 0 8 4 0]
 [0 0 0 5 8 0 0 4]]
[4 0 0 9 7 0 0 3]


TASK 5

In [194]:
def custom_substitution_number(v):
    if(v == 0):
        return 0
    if(v == 1):
        return 2
    if(v == 2):
        return 4
    if(v == 3):
        return 8
    if(v == 4):
        return 6
    if(v == 5):
        return 10
    if(v == 6):
        return 1
    if(v == 7):
        return 3
    if(v == 8):
        return 5
    if(v == 9):
        return 7
    if(v == 10):
        return 9
    
def custom_substitution(v,p):

    for i in range(8):
        v[i] = custom_substitution_number(v[i])
    
    return v


In [195]:
print(encryption(u,k,p,r,
                 substitution=custom_substitution,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator=subkey_generator_default
                 ))

[9 0 0 0 5 0 0 6]


Task 7

In [196]:
def subkey_generator_task_7(key):

    k_1 = [key[0],key[1],key[2],key[3]]
    k_2 = [key[0],key[1],key[3],key[2]]
    k_3 = [key[1],key[2],key[3],key[0]]
    k_4 = [key[0],key[3],key[1],key[2]]
    k_5 = [key[2],key[3],key[0],key[1]]
    k_6 = [key[1],key[3],key[0],key[2]]

    return [k_1,k_2,k_3,k_4,k_5,k_6]

def mod_inv(v, p):
    
    x =[]

    for num in v:
        if(num != 0):
            x.append(2*pow(num, -1, p))
        else:
            x.append(0)


    return np.array(x) # Calcola l'inverso di x modulo p

def substitution_task_7(x,p):
    x= x.tolist()
    return mod_inv(x, p)
    

In [197]:
u= np.array([1,0,0,0,0,0,0,0])
k=np.array([1,0,0,0])

p= 11

print(encryption(u,k,p,r,
                 substitution=substitution_task_7,
                 transposition=transposition,
                 subkey_sum=subkey_sum,
                 linear=linear,
                 subkey_generator = subkey_generator_task_7))

[5 0 3 2 5 2 1 1]
